## Лабораторная работа - Бустинг, бэггинг

## О задании

В этом задании вам предстоит вручную запрограммировать один из самых мощных алгоритмов машинного обучения — бустинг. Работать мы будем на двух наборах данных: многомерных данных по кредитам с kaggle и синтетических двумерных. В данных с kaggle целевая переменная показывает, вернуло ли кредит физическое лицо:

In [14]:
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, f1_score, log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier

In [15]:
#!wget  -O 'bank_data.csv' -q 'https://www.dropbox.com/s/uy27mctxo0gbuof/bank_data.csv?dl=0'

In [16]:
df = pd.read_csv('bank_data.csv')
df.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,39,admin.,married,university.degree,unknown,no,no,cellular,jul,thu,...,3,999,0,nonexistent,1.4,93.918,-42.7,4.968,5228.1,-1
1,31,services,divorced,high.school,no,yes,no,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,-1
2,34,services,divorced,high.school,no,yes,no,cellular,may,fri,...,1,999,0,nonexistent,-1.8,92.893,-46.2,1.250,5099.1,-1
3,23,admin.,single,professional.course,no,yes,no,cellular,jul,wed,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,-1
4,63,housemaid,married,basic.4y,no,no,no,cellular,aug,tue,...,1,999,0,nonexistent,-2.9,92.201,-31.4,0.838,5076.2,1


Разделим на train и test (random_state не меняем)

In [17]:
df = pd.read_csv('bank_data.csv')
categorical_columns = [
    'job',           # admin., services, etc.
    'marital',       # married, divorced, single, etc.
    'education',     # university.degree, high.school, etc.
    'default',       # unknown, no, yes
    'housing',       # no, yes
    'loan',         # no, yes
    'contact',      # cellular, telephone
    'month',        # jul, aug, sep, etc.
    'day_of_week',  # thu, mon, tue, etc.
    'poutcome'      # nonexistent, success, failure
]
numeric_columns = [
    'age',           # 39, 31, etc. (целое число)
    'duration',      # 67 (продолжительность контакта в секундах)
    'campaign',      # 3 (число контактов с этим клиентом)
    'pdays',         # 999 (дней с последнего контакта)
    'previous',      # 0 (число контактов до этой кампании)
    'emp.var.rate',  # 1.4 (показатель занятости)
    'cons.price.idx', # 93.918 (индекс потребительских цен)
    'cons.conf.idx', # -42.7 (индекс потребительской уверенности)
    'euribor3m',     # 4.968 (ставка EURIBOR 3 месяца)
    'nr.employed'    # 5228.1 (число занятых)
]
X = df.drop('y', axis=1)
y = df['y']

# Преобразуем категориальные в числовые коды
for col in categorical_columns:
    X[col] = X[col].astype('category').cat.codes

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)


Генерируем синтетические данные (seed не меняем)

In [18]:
np.random.seed(42)

num_obs = 10 ** 5
num_thresholds = 50

X_synthetic = np.random.normal(scale=3, size=[num_obs, 2])
x1_thresholds = np.random.choice(X_synthetic[:, 0], num_thresholds, False)
x2_thresholds = np.random.choice(X_synthetic[:, 1], num_thresholds, False)

gains = np.random.uniform(-0.4086, 0.5, size=[2 * num_thresholds, 1])
x1_thresholds_cond = [X_synthetic[:, 0] >= threshold for threshold in x1_thresholds]
x2_thresholds_cond = [X_synthetic[:, 1] >= threshold for threshold in x2_thresholds]

noise = np.random.uniform(-0.5, 0.5, size=num_obs)

y_synthetic_probits = np.sum(
    gains[:num_thresholds] * x1_thresholds_cond + gains[num_thresholds:] * x2_thresholds_cond, axis=0
) + noise
y_synthetic = np.sign(y_synthetic_probits)

X_train_synthetic, y_train_synthetic = X_synthetic[:int(num_obs * 0.8)], y_synthetic[:int(num_obs * 0.8)]
X_test_synthetic, y_test_synthetic = X_synthetic[int(num_obs * 0.8):], y_synthetic[int(num_obs * 0.8):]

px.histogram(x = y_synthetic_probits, nbins=100)

Некоторый полезный код для визуализации предсказаний (пригодится позже)

In [19]:
import matplotlib.pyplot as plt

def plot_predicts(model, features, targets, x_lim=[-15.0, 15.0], y_lim=[-15.0, 15.0],
                  examples_density=0.01, steps=1000, num_ticks=6, title='', mode='classification'):
    '''
    Функция для визуализации предсказаний модели на двухмерной плоскости
    param model: обученная модель классификации или регрессии для двухмерных объектов
    param features: признаки выборки (a.k.a. X)
    param targets: целевая переменная выборки (a.k.a y)
    param x_lim: пределы для x
    param y_lim: пределы для y
    param examples_density: доля выборки, которая будет нарисована
    param steps: частота разбиения плоскости
    param num_ticks: число подписей на графике
    param title: заголовок графика
    param mode: режим 'classification' - вероятности положительного класса
                режим 'regression' - вещественная целевая переменная
    '''
    
    mask = np.random.choice([True, False], size=features.shape[0], 
                            p=[examples_density, 1.0 - examples_density])
    features_x = (features[mask, 0] - x_lim[0]) / (x_lim[1] - x_lim[0]) * steps
    features_y = (features[mask, 1] - y_lim[0]) / (y_lim[1] - y_lim[0]) * steps
    
    xs = np.linspace(x_lim[0], x_lim[1], steps)
    ys = np.linspace(y_lim[0], y_lim[1], steps)
    
    xs, ys = np.meshgrid(xs, ys)
    grid = np.stack([xs.flatten(), ys.flatten()], axis=1)
    if mode == 'classification':
        predicts = model.predict_proba(grid)[:, 1].reshape(steps, steps)
        values = (targets[mask] == 1).astype(np.float)
    elif mode == 'regression':
        predicts = model.predict(grid).reshape(steps, steps)
        values = targets[mask]
    else:
        raise ValueError('Unknown mode')
    
    plt.figure(figsize=(10, 10))
    plt.imshow(predicts, origin='lower')
    plt.scatter(features_x, features_y, c=values, edgecolors='white', linewidths=1.5)
    plt.colorbar()
    
    plt.xticks(np.linspace(0, steps, num_ticks), np.linspace(x_lim[0], x_lim[1], num_ticks))
    plt.yticks(np.linspace(0, steps, num_ticks), np.linspace(y_lim[0], y_lim[1], num_ticks))
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title(title)
    plt.grid()
    plt.show()

#### 1. Реализуйте бустинг для задачи бинарной классификации.

Поскольку градиентный бустинг обучается через последовательное создание моделей, может получиться так, что оптимальная с точки зрения генерализации модель будет получена на промежуточной итерации. Обычно для контроля такого поведения в методе `fit` передается также валидационная выборка, по которой можно оценивать общее качество модели в процессе обучения (желательно делать это каждую итерацию, но если ваша имплементация слишком медленная или ваше железо не тянет, можно делать это реже). Кроме того, нет смысла обучать действительно глубокую модель на 1000 деревьев и больше, если оптимальный ансамбль получился, к примеру, на 70 итерации и в течение какого-то количества итераций не улучшился - поэтому мы также задействуем early stopping при отсутствии улучшений в течение некоторого числа итераций.

In [20]:
class Boosting:
    
    def __init__(
        self,
        base_model_class=DecisionTreeRegressor,
        base_model_params: dict={'max_features': 0.1},
        n_estimators: int=10,
        learning_rate: float=0.1,
        subsample: float=0.3,
        random_seed: int=228,
        custom_loss: list or tuple=None,
        use_best_model: bool=False,
        n_iter_early_stopping: int=None
    ):
        
        # Класс базовой модели
        self.base_model_class = base_model_class
        # Параметры для инициализации базовой модели
        self.base_model_params = base_model_params
        # Число базовых моделей
        self.n_estimators = n_estimators
        # Длина шага (которая в лекциях обозначалась через eta)
        self.learning_rate = learning_rate
        # Доля объектов, на которых обучается каждая базовая модель
        self.subsample = subsample
        # seed для бутстрапа, если хотим воспроизводимость модели
        self.random_seed = random_seed
        # Использовать ли при вызове predict и predict_proba лучшее
        # с точки зрения валидационной выборки число деревьев в композиции
        self.use_best_model = use_best_model
        # число итераций, после которых при отсутствии улучшений на валидационной выборке обучение завершается
        self.n_iter_early_stopping = n_iter_early_stopping
        
        # Плейсхолдер для нулевой модели
        self.initial_model_pred = None
        
        # Список для хранения весов при моделях
        self.gammas = []
        
        # Создаем список базовых моделей
        self.models = [self.base_model_class(**self.base_model_params) for _ in range(self.n_estimators)]
        
        # Если используем свою функцию потерь, ее нужно передать как список из loss-a и его производной
        if custom_loss is not None:
            self.loss_fn, self.loss_derivative = custom_loss
        else:
            self.sigmoid = lambda z: 1 / (1 + np.exp(-z))
            self.loss_fn = lambda y, z: -np.log(self.sigmoid(y * z)).mean()
            self.loss_derivative = lambda y, z: -y * self.sigmoid(-y * z)
        
        
    def _fit_new_model(self, X: np.ndarray, y: np.ndarray or list, n_model: int):
        """
        Функция для обучения одной базовой модели бустинга
        :param X: матрица признаков
        :param y: вектор целевой переменной
        :param n_model: номер модели, которую хотим обучить
        """
        rng = np.random.RandomState(self.random_seed + n_model)
        n_samples = X.shape[0]
        subsample_size = int(self.subsample * n_samples)
        indices = rng.choice(n_samples, size=subsample_size, replace=False)
        
        X_subsample = X.iloc[indices] if hasattr(X, 'iloc') else X[indices]
        y_subsample = y.iloc[indices] if hasattr(y, 'iloc') else y[indices]
        
        # 2. Обучаем базовую модель на подвыборке
        model = self.models[n_model]
        model.fit(X_subsample, y_subsample)
        
        # 3. Возвращаем предсказания обученной модели на всей выборке
        return model.predict(X)
        
        
    def _fit_initial_model(self, X, y):
        """
        Функция для построения нулевой (простой) модели. Не забудьте взять логарифм, потому что сигмоида применяется
        уже к сумме предсказаний базовых моделей, а не к каждому предсказанию каждой модели по отдельности.
        Подойдёт константная модель, возвращающая самый популярный класс,
        но если хотите, можете сделать что-нибудь посложнее.
        """
        # Преобразуем y в numpy array для удобства
        y = np.array(y)
        
        # 1. Вычисляем оптимальное начальное предсказание
        p_positive = np.mean(y == 1)  # доля положительных примеров
        p_negative = 1 - p_positive   # доля отрицательных примеров
        
        # 2. Чтобы избежать деления на 0 и логарифма от 0
        epsilon = 1e-15
        p_positive = np.clip(p_positive, epsilon, 1 - epsilon)
        p_negative = np.clip(p_negative, epsilon, 1 - epsilon)
        
        # 3. Вычисляем логарифм отношения вероятностей
        initial_prediction = np.log(p_positive / p_negative)
        
        # 4. Сохраняем начальное предсказание
        self.initial_model_pred = initial_prediction
        
        # 5. Возвращаем массив начальных предсказаний для всех объектов
        return np.full(X.shape[0], initial_prediction)
    
    
    def _find_optimal_gamma(self, y: np.ndarray or list, old_predictions: np.ndarray,
                            new_predictions: np.ndarray, boundaries: tuple or list=(0.01, 1)):
        """
        Функция для поиска оптимального значения параметра gamma (коэффициент перед новой базовой моделью).
        :param y: вектор целевой переменной
        :param old_predictions: вектор суммы предсказаний предыдущих моделей (до сигмоиды)
        :param new_predictions: вектор суммы предсказаний новой модели (после сигмоиды)
        :param boudnaries: в каком диапазоне искать оптимальное значение ɣ (array-like объект из левой и правой границ)
        """
        # Определеяем начальные лосс и оптимальную гамму
        loss, optimal_gamma = self.loss_fn(y, old_predictions), 0
        # Множество, на котором будем искать оптимальное значение гаммы
        gammas = np.linspace(*boundaries, 100)
        # Простым перебором ищем оптимальное значение
        for gamma in gammas:
            predictions = old_predictions + gamma * new_predictions
            gamma_loss = self.loss_fn(y, predictions)
            if gamma_loss < loss:
                optimal_gamma = gamma
                loss = gamma_loss
        
        return optimal_gamma
        
        
    def fit(self, X, y, eval_set=None):
        """
        Функция для обучения всей модели бустинга
        :param X: матрица признаков
        :param y: вектор целевой переменной
        :eval_set: кортеж (X_val, y_val) для контроля процесса обучения или None, если контроль не используется
        """
        # 1. Инициализируем список для хранения истории ошибок (для ранней остановки)
        if eval_set is not None:
            X_val, y_val = eval_set
            self.val_errors = []
            self.best_iteration = 0
            self.best_loss = float('inf')
        
        # 2. Обучаем начальную модель и получаем начальные предсказания
        current_predictions = self._fit_initial_model(X, y)
        
        # 3. Основной цикл обучения базовых моделей
        for i in range(self.n_estimators):
            # 4. Вычисляем антиградиент (производная функции потерь)
            # Это то, что мы будем предсказывать следующей моделью
            gradients = -self.loss_derivative(y, current_predictions)
            
            # 5. Обучаем новую базовую модель для предсказания антиградиента
            new_predictions = self._fit_new_model(X, gradients, i)
            
            # 6. Находим оптимальный шаг gamma для этой модели
            gamma = self._find_optimal_gamma(y, current_predictions, new_predictions)
            self.gammas.append(gamma)
            
            # 7. Обновляем текущие предсказания
            current_predictions += self.learning_rate * gamma * new_predictions
            
            # 8. Контроль качества на валидационной выборке (если есть)
            if eval_set is not None:
                val_predictions = self._predict_raw(X_val)  # Предсказания в сыром виде (до сигмоиды)
                val_loss = self.loss_fn(y_val, val_predictions)
                self.val_errors.append(val_loss)
                
                # Проверяем, улучшилась ли модель
                if val_loss < self.best_loss:
                    self.best_loss = val_loss
                    self.best_iteration = i
                
                # Ранняя остановка
                if (self.n_iter_early_stopping is not None and 
                    i - self.best_iteration >= self.n_iter_early_stopping):
                    print(f"Early stopping at iteration {i}")
                    # Обрезаем списки моделей и гамм
                    self.models = self.models[:i+1]
                    self.gammas = self.gammas[:i+1]
                    break
        
        
    def predict(self, X: np.ndarray):
        """
        Функция для предсказания классов обученной моделью бустинга
        :param X: матрица признаков
        """
        # 1. Используем predict_proba для получения вероятностей
        probabilities = self.predict_proba(X)
        
        # 2. Берем вероятности класса 1 (второй столбец)
        proba_positive = probabilities[:, 1]
        classes = np.where(proba_positive > 0.5, 1, -1)
        return classes
        
    def _predict_raw(self, X: np.ndarray):
        """
        Вспомогательная функция для получения сырых предсказаний (до сигмоиды)
        :param X: матрица признаков
        """
        # 1. Начинаем с начального предсказания
        predictions = np.full(X.shape[0], self.initial_model_pred)
        
        # 2. Определяем, сколько моделей использовать
        if self.use_best_model and hasattr(self, 'best_iteration'):
            n_models_to_use = self.best_iteration + 1
        else:
            n_models_to_use = len(self.models)
        
        # 3. Добавляем взвешенные предсказания всех базовых моделей
        for i in range(n_models_to_use):
            # Проверяем, что модель была обучена (может быть None при ранней остановке)
            if self.models[i] is not None:
                model_pred = self.models[i].predict(X)
                predictions += self.learning_rate * self.gammas[i] * model_pred
        
        return predictions

    def predict_proba(self, X: np.ndarray):
        """
        Функция для предсказания вероятностей классов обученной моделью бустинга
        :param X: матрица признаков
        """
            # 1. Получаем "сырые" предсказания (сумму до сигмоиды)
        raw_predictions = self._predict_raw(X)
        
        # 2. Применяем сигмоиду для получения вероятности класса 1
        proba_positive = self.sigmoid(raw_predictions)
        
        # 3. Для бинарной классификации возвращаем матрицу [P(y=-1), P(y=1)]
        # P(y=-1) = 1 - P(y=1)
        proba_negative = 1 - proba_positive
        
        # 4. Объединяем в матрицу формы (n_samples, 2)
        probabilities = np.column_stack([proba_negative, proba_positive])
        
        return probabilities

    @property
    def feature_importances_(self):
        """
        Для бонусного задания номер 5.
        Функция для вычисления важностей признаков.
        Вычисление должно проводиться после обучения модели
        и быть доступно атрибутом класса. 
        """
        # Your code here ╰( ͡° ͜ʖ ͡° )つ──☆*:

#### Тест для вашей имплементации. Если класс написан правильно, две следующие ячейки должна отработать без ошибок и относительно быстро (у автора задания 2 и 0.2 секунд соответственно, accuracy 0.911 и 0.879 соответственно). Если у вас получилось качество выше указанного — отлично!

In [21]:
%%time

boosting = Boosting()
boosting.fit(X_train_synthetic, y_train_synthetic)
# Без разницы, выдает эта строка классы или вероятности
preds = np.round(boosting.predict(X_test_synthetic) > 0.5)
assert accuracy_score((y_test_synthetic == 1), np.round(preds)) > 0.9

CPU times: total: 1.61 s
Wall time: 1.67 s


In [22]:
%%time

boosting = Boosting()
boosting.fit(X_train.select_dtypes(['int64', 'float64']).values, y_train.values)
# Без разницы, выдает эта строка классы или вероятности
preds = np.round(boosting.predict(X_test.select_dtypes(['int64', 'float64']).values) > 0.5)
assert accuracy_score((y_test.values == 1), np.round(preds)) > 0.87

CPU times: total: 125 ms
Wall time: 138 ms


#### 2. Сравните результаты вашей имплементации бустинга с указанными ниже базовыми моделями на обоих датасетах и ответьте на вопросы. Разумеется, надо измерять качество на тестовых данных. 

Варианты для базовой модели (разумеется, не надо их программировать самостоятельно, берите нужные классы из sklearn):

- Решающее дерево глубины 6
- Случайный лес (число деревьев — на ваше усмотрение, только не слишком мало)
- Линейная регрессия

Вопросы:

1) Какая из моделей имеет оптимальное качество? С чем это связано?

2) Какая из моделей сильнее переобучается? Есть ли преимущества от использования ранней остановки и обрезания бустинга до лучшей модели?

3) Работает ли бустинг над линейными регрессиями лучше, чем одна логистическая регрессия? Как объяснить этот результат?

4) Визуализируйте предсказания моделей на синтетическом датасете (для этого можете воспользоваться вспомогательной функцией plot_predicts). Чем отличаются картинки, которые получаются у разных алгоритмов? Сделайте выводы.

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

# Создаём бустинг с деревьями глубины 6
boostingTree = Boosting(
    base_model_class=DecisionTreeRegressor,
    base_model_params={
        'max_depth': 6,
        'random_state': 42
    },
    n_estimators=100,
    learning_rate=0.1,
    subsample=0.8
)

# Инициализация бустинга со случайным лесом в качестве базовой модели
boosting_rf = Boosting(
    base_model_class=RandomForestRegressor,
    base_model_params={
        'n_estimators': 50,           # Число деревьев в лесе
        'max_depth': 3,               # Глубина деревьев
        'min_samples_split': 20,      # Минимальное samples для разделения
        'min_samples_leaf': 10,       # Минимальное samples в листе
        'max_features': 'sqrt',       # Число признаков для поиска разделения
        'random_state': 42,           # Для воспроизводимости
        'n_jobs': -1                  # Использовать все ядра процессора
    },
    n_estimators=100,                 # Число базовых моделей в бустинге
    learning_rate=0.1,                # Темп обучения
    subsample=0.8,                    # Доля объектов для обучения каждой модели
    random_seed=42,
    use_best_model=True,
    n_iter_early_stopping=10
)

# Инициализация бустинга с линейной регрессией в качестве базовой модели
boosting_lr = Boosting(
    base_model_class=LinearRegression,
    base_model_params={
        'fit_intercept': True,        # Подгонять intercept
        'copy_X': True,               # Копировать данные
        'n_jobs': -1                  # Использовать все ядра
    },
    n_estimators=100,
    learning_rate=0.05,               # Меньший learning_rate для линейных моделей
    subsample=0.9,                    # Большая доля выборки для стабильности
    random_seed=42,
    use_best_model=True,
    n_iter_early_stopping=15
)

boosters = [
    boostingTree,
    boosting_rf,
    boosting_lr
]

for booster in boosters:
    booster.fit(X_train_synthetic, y_train_synthetic)
    y_pred_synthetic = booster.predict(X_test_synthetic)
    accuracy_synthetic = (y_pred_synthetic == y_test_synthetic).mean()
    print(f'Точность модели на синтетических данных: {accuracy_synthetic}')

    booster.fit(X_train, y_train)
    y_pred = booster.predict(X_test)
    accuracy = (y_pred == y_test).mean()
    print(f'Точность модели на настоящих данных: {accuracy}')

Точность модели на синтетических данных: 0.91955
Точность модели на настоящих данных: 0.891882183908046
Точность модели на синтетических данных: 0.89805
Точность модели на настоящих данных: 0.8703304597701149
Точность модели на синтетических данных: 0.689
Точность модели на настоящих данных: 0.8516522988505747


#### 3. Мы разобрались с бустингом, теперь интересно посмотреть на комбинации моделей. Сравните результаты следующих моделей на обоих датасетах и ответьте на вопросы. Разумеется, надо измерять качество на тестовых данных.

Используйте логистическую регрессию, случайный лес и BaggingClassifier из sklearn.

- Случайный лес
- Бэггинг на деревьях (поставьте для базовых деревьев min_samples_leaf=1)
- Бэггинг на деревьях с обучением каждого дерева на подмножестве признаков (`max_features` около 0.6 в BaggingClassifier)
- Бэггинг, у которого базовой моделью является бустинг с большим числом деревьев (> 100)
- Бэггинг на логистических регрессиях

1) Какая из моделей имеет лучшее качество? С чем это связано?

2) Какая из моделей сильнее всего переобучается? Помогает ли бустингу ранняя остановка? 

3) Исправляет ли бэггинг переобученность бустинга с большим числом деревьев?

4) Что лучше: случайный лес или бэггинг на деревьях с сэмплированием признаков?

5) Если использовать деревья в качестве базового алгоритма, что лучше — бэггинг или бустинг? С чем это связано?

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import time

# Создаем словарь для хранения результатов
results = {}

# 1. Случайный лес
print("1. Обучаем Случайный лес...")
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)
start_time = time.time()
rf.fit(X_train, y_train)
rf_time = time.time() - start_time
rf_pred = rf.predict(X_test)
rf_proba = rf.predict_proba(X_test)[:, 1]

results['Random Forest'] = {
    'accuracy': accuracy_score(y_test, rf_pred),
    'f1': f1_score(y_test, rf_pred, average='weighted'),
    'roc_auc': roc_auc_score(y_test, rf_proba),
    'time': rf_time
}

start_time = time.time()
rf.fit(X_train_synthetic, y_train_synthetic)
rf_time = time.time() - start_time
rf_pred = rf.predict(X_test_synthetic)
rf_proba = rf.predict_proba(X_test_synthetic)[:, 1]

results['Random Forest Synthetic'] = {
    'accuracy': accuracy_score(y_test_synthetic, rf_pred),
    'f1': f1_score(y_test_synthetic, rf_pred, average='weighted'),
    'roc_auc': roc_auc_score(y_test_synthetic, rf_proba),
    'time': rf_time
}

# 2. Бэггинг на деревьях (min_samples_leaf=1)
print("2. Обучаем Бэггинг на деревьях...")
bagging_trees = BaggingClassifier(
    estimator=DecisionTreeClassifier(min_samples_leaf=1, random_state=42),
    n_estimators=50,
    random_state=42,
    n_jobs=-1
)
start_time = time.time()
bagging_trees.fit(X_train, y_train)
bagging_trees_time = time.time() - start_time
bagging_trees_pred = bagging_trees.predict(X_test)
bagging_trees_proba = bagging_trees.predict_proba(X_test)[:, 1]

results['Bagging Trees (min_samples_leaf=1)'] = {
    'accuracy': accuracy_score(y_test, bagging_trees_pred),
    'f1': f1_score(y_test, bagging_trees_pred, average='weighted'),
    'roc_auc': roc_auc_score(y_test, bagging_trees_proba),
    'time': bagging_trees_time
}

start_time = time.time()
bagging_trees.fit(X_train_synthetic, y_train_synthetic)
bagging_trees_time = time.time() - start_time
bagging_trees_pred = bagging_trees.predict(X_test_synthetic)
bagging_trees_proba = bagging_trees.predict_proba(X_test_synthetic)[:, 1]

results['Bagging Trees Synthetic'] = {
    'accuracy': accuracy_score(y_test_synthetic, bagging_trees_pred),
    'f1': f1_score(y_test_synthetic, bagging_trees_pred, average='weighted'),
    'roc_auc': roc_auc_score(y_test_synthetic, bagging_trees_proba),
    'time': bagging_trees_time
}


# 3. Бэггинг на деревьях с подмножеством признаков (max_features=0.6)
print("3. Обучаем Бэггинг с подмножеством признаков...")
bagging_trees_features = BaggingClassifier(
    estimator=DecisionTreeClassifier(random_state=42),
    n_estimators=50,
    max_features=0.6,
    random_state=42,
    n_jobs=-1
)
start_time = time.time()
bagging_trees_features.fit(X_train, y_train)
bagging_trees_features_time = time.time() - start_time
bagging_trees_features_pred = bagging_trees_features.predict(X_test)
bagging_trees_features_proba = bagging_trees_features.predict_proba(X_test)[:, 1]

results['Bagging Trees (max_features=0.6)'] = {
    'accuracy': accuracy_score(y_test, bagging_trees_features_pred),
    'f1': f1_score(y_test, bagging_trees_features_pred, average='weighted'),
    'roc_auc': roc_auc_score(y_test, bagging_trees_features_proba),
    'time': bagging_trees_features_time
}

start_time = time.time()
bagging_trees_features.fit(X_train_synthetic, y_train_synthetic)
bagging_trees_features_time = time.time() - start_time
bagging_trees_features_pred = bagging_trees_features.predict(X_test_synthetic)
bagging_trees_features_proba = bagging_trees_features.predict_proba(X_test_synthetic)[:, 1]

results['Bagging Trees (max_features=0.6) Synthetic'] = {
    'accuracy': accuracy_score(y_test_synthetic, bagging_trees_features_pred),
    'f1': f1_score(y_test_synthetic, bagging_trees_features_pred, average='weighted'),
    'roc_auc': roc_auc_score(y_test_synthetic, bagging_trees_features_proba),
    'time': bagging_trees_features_time
}

# 4. Бэггинг на бустинге (нужно реализовать наш бустинг)
print("4. Обучаем Бэггинг на бустинге...")

# Создаем базовый бустинг
base_boosting = GradientBoostingClassifier(
    n_estimators=150,  # > 100 как требуется
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)

bagging_boosting = BaggingClassifier(
    estimator=base_boosting,
    n_estimators=10,  # 10 бустингов в бэггинге
    random_state=42,
    n_jobs=-1
)

start_time = time.time()
bagging_boosting.fit(X_train.values, y_train.values)  # преобразуем в numpy для совместимости
bagging_boosting_time = time.time() - start_time
bagging_boosting_pred = bagging_boosting.predict(X_test.values)
bagging_boosting_proba = bagging_boosting.predict_proba(X_test.values)[:, 1]

results['Bagging on Boosting'] = {
    'accuracy': accuracy_score(y_test, bagging_boosting_pred),
    'f1': f1_score(y_test, bagging_boosting_pred, average='weighted'),
    'roc_auc': roc_auc_score(y_test, bagging_boosting_proba),
    'time': bagging_boosting_time
}

start_time = time.time()
bagging_boosting.fit(X_train_synthetic, y_train_synthetic)  # преобразуем в numpy для совместимости
bagging_boosting_time = time.time() - start_time
bagging_boosting_pred = bagging_boosting.predict(X_test_synthetic)
bagging_boosting_proba = bagging_boosting.predict_proba(X_test_synthetic)[:, 1]

results['Bagging on Boosting Synthetic'] = {
    'accuracy': accuracy_score(y_test_synthetic, bagging_boosting_pred),
    'f1': f1_score(y_test_synthetic, bagging_boosting_pred, average='weighted'),
    'roc_auc': roc_auc_score(y_test_synthetic, bagging_boosting_proba),
    'time': bagging_boosting_time
}

# 5. Бэггинг на логистических регрессиях
print("5. Обучаем Бэггинг на логистических регрессиях...")

# Масштабируем данные для логистической регрессии
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_synthetic_scaled = scaler.fit_transform(X_train_synthetic)
X_test_synthetic_scaled = scaler.transform(X_test_synthetic)

bagging_lr = BaggingClassifier(
    estimator=LogisticRegression(random_state=42, max_iter=1000),
    n_estimators=50,
    random_state=42,
    n_jobs=-1
)
start_time = time.time()
bagging_lr.fit(X_train_scaled, y_train)
bagging_lr_time = time.time() - start_time
bagging_lr_pred = bagging_lr.predict(X_test_scaled)
bagging_lr_proba = bagging_lr.predict_proba(X_test_scaled)[:, 1]

results['Bagging on Logistic Regression'] = {
    'accuracy': accuracy_score(y_test, bagging_lr_pred),
    'f1': f1_score(y_test, bagging_lr_pred, average='weighted'),
    'roc_auc': roc_auc_score(y_test, bagging_lr_proba),
    'time': bagging_lr_time
}

start_time = time.time()
bagging_lr.fit(X_train_synthetic_scaled, y_train_synthetic)
bagging_lr_time = time.time() - start_time
bagging_lr_pred = bagging_lr.predict(X_test_synthetic_scaled)
bagging_lr_proba = bagging_lr.predict_proba(X_test_synthetic_scaled)[:, 1]

results['Bagging on Logistic Regression Synthetic'] = {
    'accuracy': accuracy_score(y_test_synthetic, bagging_lr_pred),
    'f1': f1_score(y_test_synthetic, bagging_lr_pred, average='weighted'),
    'roc_auc': roc_auc_score(y_test_synthetic, bagging_lr_proba),
    'time': bagging_lr_time
}

1. Обучаем Случайный лес...
2. Обучаем Бэггинг на деревьях...
3. Обучаем Бэггинг с подмножеством признаков...
4. Обучаем Бэггинг на бустинге...
5. Обучаем Бэггинг на логистических регрессиях...


#### 4. Сравните на этих данных любую из трёх популярных имплементаций градиентного бустинга (xgboost, lightgbm, catboost) с вашей реализацией. Подберите основные гиперпараметры (число деревьев, длина шага, глубина дерева/число листьев) для обоих методов. Получилось ли у вас победить библиотечные реализации на тестовых данных?

In [25]:
import xgboost as xgb

boosting_results = {}

# 1. Наша реализация
print("1. Обучаем нашу реализацию бустинга...")
our_boosting = Boosting(
    base_model_class=DecisionTreeRegressor,
    base_model_params={
        'max_depth': 6,
        'random_state': 42
    },
    n_estimators=100,
    learning_rate=0.1,
    subsample=0.8
)

start_time = time.time()
our_boosting.fit(X_train, y_train)
our_time = time.time() - start_time
our_pred = our_boosting.predict(X_test)
our_proba = our_boosting.predict_proba(X_test)[:, 1]

boosting_results['Our Boosting'] = {
    'accuracy': accuracy_score(y_test, our_pred),
    'f1': f1_score(y_test, our_pred, average='weighted'),
    'roc_auc': roc_auc_score(y_test, our_proba),
    'time': our_time
}

start_time = time.time()
our_boosting.fit(X_train_synthetic, y_train_synthetic)
our_time = time.time() - start_time
our_pred = our_boosting.predict(X_test_synthetic)
our_proba = our_boosting.predict_proba(X_test_synthetic)[:, 1]

boosting_results['Our Boosting Synthetic'] = {
    'accuracy': accuracy_score(y_test_synthetic, our_pred),
    'f1': f1_score(y_test_synthetic, our_pred, average='weighted'),
    'roc_auc': roc_auc_score(y_test_synthetic, our_proba),
    'time': our_time
}

print("2. Обучаем XGBoost...")

xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    random_state=42,
    eval_metric='logloss',
    n_jobs=-1
)

start_time = time.time()
y_train_clipped = y_train.clip(0, 1)
xgb_model.fit(X_train, y_train_clipped)
xgb_time = time.time() - start_time

xgb_pred = xgb_model.predict(X_test)
xgb_pred = np.where(xgb_pred == 0, -1, xgb_pred)
xgb_proba = xgb_model.predict_proba(X_test)[:, 1]

boosting_results['XGBoost'] = {
    'accuracy': accuracy_score(y_test, xgb_pred),
    'f1': f1_score(y_test, xgb_pred, average='weighted'),
    'roc_auc': roc_auc_score(y_test, xgb_proba),
    'time': xgb_time
}

start_time = time.time()
y_train_clipped = y_train_synthetic.clip(0, 1)
xgb_model.fit(X_train_synthetic, y_train_clipped)
xgb_time = time.time() - start_time

xgb_pred = xgb_model.predict(X_test_synthetic)
xgb_pred = np.where(xgb_pred == 0, -1, xgb_pred)
xgb_proba = xgb_model.predict_proba(X_test_synthetic)[:, 1]

boosting_results['XGBoost Synthetic'] = {
    'accuracy': accuracy_score(y_test_synthetic, xgb_pred),
    'f1': f1_score(y_test_synthetic, xgb_pred, average='weighted'),
    'roc_auc': roc_auc_score(y_test_synthetic, xgb_proba),
    'time': xgb_time
}


res = pd.DataFrame(boosting_results).T.round(4)
res

1. Обучаем нашу реализацию бустинга...
2. Обучаем XGBoost...


,accuracy,f1,roc_auc,time
Our Boosting,0.8919,0.8916,0.9488,4.5622
Our Boosting Synthetic,0.9196,0.9195,0.9810,19.6501
XGBoost,0.8897,0.8895,0.9479,0.0846
XGBoost Synthetic,0.9216,0.9216,0.9827,0.2243
